In [106]:
import graphviz as gv
# Hab das mal bei mir installiert für die Darstellung
import os

import csv, re
import numpy as np
from typing import List

In [107]:
rows = []
with open("../../example-structure.csv", "r") as f:
    reader = csv.reader(f)
    rows = [row for row in reader if row != []]
print(rows)

[['Semaphore', ' 12', ' 0'], ['Semaphore', ' 13', ' 0'], ['Semaphore', ' 36', ' 0'], ['Semaphore', ' 24', ' 0'], ['Semaphore', ' 46', ' 0'], ['Semaphore', ' 65', ' 0 '], ['Semaphore', ' 5ab', ' 0'], ['Semaphore', ' 5ba', ' 1'], ['Semaphore', ' 51', ' 1'], ['Mutex', ' 234'], ['Mutex', ' 345'], ['Activity', ' 1', ' 8', ' 51', ' 12;13', ' x'], ['Activity', ' 2', ' 8', ' 12', ' 24', ' 234;345'], ['Activity', ' 3', ' 8', ' 13', ' 36', ' 234'], ['Activity', ' 4', ' 8', ' 24', ' 46', ' 234'], ['Activity', ' 5a', ' 8', ' 65;5ba', ' 5ab', ' x'], ['Activity', ' 5b', ' 8', ' 5ab', ' 51;5ba', ' x'], ['Activity', ' 6', ' 8', ' 46:36', ' 65', ' x'], ['Task', ' 1', ' 1'], ['Task', ' 2', ' 2'], ['Task', ' 3', ' 3'], ['Task', ' 4', ' 4'], ['Task', ' 5', ' 5a;5b'], ['Task', ' 6', ' 6']]


In [37]:
class Semaphore():
    def __init__(self, name, value=0):
        _name = name
        _value = value
class Activity():
    def __init__(self, name, duration, incoming_semaphores, outgoing_semaphores):
         _name = name
         _duration = duration
         _incoming_semaphores = incoming_semaphores
         _outgoing_semaphores = outgoing_semaphores
class Task():
    def __init__(self, task_name, activity_list: List):
         _name = task_name
         _activities = activity_list
    def get_name(self) -> str:
        return  _name
    
    def get_activities(self) -> List:
        return  _activities

class Mutex():
    def __init__(self, activity_list: List):
         _activity_list = activity_list
         _reserved = False

_semaphores = []
_activities = []
_tasks = []
_mutexes = []

def find_activities(activities) -> List[Activity]:
    found_activities = []
    for activty in activities.split(";"):
        found_activities.append(find_in_array(_activities, activty))
    return found_activities
def parse(rows):
        global _semaphores, _activities, _tasks, _mutexes
        for object in rows:
            if object[0] == "Task":
                included_activities = find_activities(object[2])
                _tasks.append(Task(object[1], included_activities))
                print("Task created")
            elif object[0] == "Activity":
                incoming_semaphores = find_semaphores(object[3])
                outgoing_semaphores = find_semaphores(object[4])
                _activities.append(Activity(object[1], object[2], incoming_semaphores, outgoing_semaphores))
                print("Activity created")
            elif object[0] == "Semaphore":
                _semaphores.append(Semaphore(object[1].strip(), object[2].strip()))
                print("Semaphore created")
            #elif object[0] == "Mutex":
            #    connected_semaphores = find_semaphores(object[1])
            #    _mutexes.append(Mutex(connected_semaphores))
            #    print("Mutex created")

def find_semaphores( semaphores) -> List[Semaphore]:
    found_semaphores = []
    for semaphore in semaphores.split(";"):
        if semaphore.count(":") > 0:
            found_semaphore_relation = []
            # Do something if argument is a numpy array
            for related_semaphore in semaphore.split(":"):
                found_semaphore_relation.append(find_in_array(_semaphores, related_semaphore.strip()))
            found_semaphores.append(found_semaphore_relation)
        else:
            # Do something if argument is a string
            found_semaphores.append(find_in_array(_semaphores, semaphore))
    return found_semaphores

def find_in_array(array, name):
    return [element for element in array if element._name == name.strip()]


parse(rows)
print("Finihsed")

Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Semaphore created
Activity created
Activity created
Activity created
Activity created
Activity created
Activity created
Activity created
Activity created
Task created
Task created
Task created
Task created
Task created
Task created
Finihsed


In [123]:
class Semaphore():
    def __init__(self, name, state=0) -> None:
        self._name = name
        self._state = state

        self._combined = []
        self._actuators = []
        self._waiting_activities = []

    def get_name(self) -> str:
        return self._name
    def get_actuators(self):
        return self._actuators
    
    def add_to_actuators(self, activity):
        self._actuators.append(activity)
    
    def get_waiting_activities(self):
        return self._waiting_activities
    
    def add_to_waiting_activities(self, activity):
        self._waiting_activities.append(activity)

    def get_state(self) -> int:
        return self._state
    
    def set_state(self, value) -> int:
        self._state = value

    def set_combined(self, value):
        self._combined = []
        self._combined.extend(value)

    def get_combined(self):
        return self._combined
        
    def reserve(self) -> bool:
        if self._state > 0:
            self._state -= 1
            return True
        else:
            return False

    def release(self):
        self._state += 1

class Activity():
    def __init__(self, name, actvity_duration, incoming_semaphores, outgoing_semaphores, relevant_mutexes, active=False) -> None:
        self._name = name
        self._duration = actvity_duration
        self._temp_duration = actvity_duration
        self._incoming_semaphores = incoming_semaphores
        self._outgoing_semaphores = outgoing_semaphores
        self._relevant_mutexes = relevant_mutexes
        self._active = active
        self._task = None

    def get_name(self) -> str:
        return self._name

    def get_duration(self) -> str:
        return self._duration
    
    def set_task(self, task):
        if self._task is not None:
            print("ERROR: Activity already has a task assigned to it")
            return
        self._task = task

    def get_task(self):
        return self._task

    def get_active(self):
        return self._active

    def get_incoming_semaphores(self) -> List:
        return self._incoming_semaphores

    def get_outgoing_semaphores(self) -> List:
        return self._outgoing_semaphores

    def get_relevant_mutexes(self) -> List:
        return self._relevant_mutexes
    
class Task():
    def __init__(self, name, activity_list: List):
        self._name = name
        self._activities = activity_list

    def get_name(self) -> str:
        return  self._name
    
    def get_activities(self) -> List:
        return  self._activities

class Mutex():
    def __init__(self, name):
        self._name = name
        self._activity_list = []
        self._reserved = False
    
    def get_name(self) -> str:
        return self._name

    def get_state(self) -> bool:
        return self._reserved

    def get_activity_list(self):
        return self._activity_list
    
    def add_to_activity_list(self, activity):
        self._activity_list.append(activity)

    def reserve(self):
        self._reserved = True

    def release(self):
        self._reserved = False

_semaphores = []
_activities = []
_tasks = []
_mutexes = []

def check_file_structure(rows: List[List[str]]) -> bool:
        print("Checking structure...")
        structure_is_good = True
        allowed_items = ["Semaphore", "Mutex", "Activity", "Task"]
        scanned_items = []
        current_index = 0
        for row in rows:
            try:
                if row[0] != allowed_items[current_index]:
                    current_index += 1
                if row[0] not in scanned_items:
                    scanned_items.append(row[0])
                if current_index > 3:
                    print("ERROR: Wrong file structure. Check if the file lists the Semaphores, Activities, Mutexes and Tasks in the correct order. Other items are not allowed.")
                    structure_is_good = False
            except IndexError:
                rows.remove(row)
                print("Empty Line (IndexError) resolved by deletion: " + str(row))
            
            if row[0] == "Semaphore" and len(row) != 3:
                print("ERROR: Wrong Semaphore structure. Check if the Semaphores have the correct amount of columns. 'Semaphore', 'Name', 'Activated'")
                structure_is_good = False
            if row[0] == "Mutex" and len(row) != 2:
                print("ERROR: Wrong Mutex structure. Check if the Mutexes have the correct amount of columns. 'Mutex', 'Name'")
                structure_is_good = False
            if row[0] == "Activity" and (len(row) != 5 and len(row) != 6):
                print("ERROR: Wrong Activity structure. Check if the Activities have the correct amount of columns. 'Activity', 'Duration', 'Name', 'Ingoing Semaphores', 'Outgoing Semaphores', 'Mutexes'")
                structure_is_good = False
            if row[0] == "Task" and len(row) !=3:
                print("ERROR: Wrong Task structure. Check if the Tasks have the correct amount of columns. 'Task', 'Name', 'Activities'")
                structure_is_good = False
        if scanned_items != allowed_items:
            print("ERROR: Wrong file structure. Check if the file lists the Semaphores, Activities, Mutexes and Tasks in the correct order. Other items are not allowed.")
            structure_is_good = False
        if structure_is_good:
            print("Structure is correct.")
        else:
            print("Structure is not correct.")
        return structure_is_good

# rows from FileReader
def parse(rows):
    data_restructuring(rows)
    if not check_file_structure(rows):
        return False
    for object in rows:
        if object[0] == "Task":  
            included_activities =  find_activities(object[2])
            _tasks.append(Task(object[1], included_activities))
            print(f"Task created: {object[1]}")
        elif object[0] == "Activity":
            incoming_semaphores =  find_semaphores(object[3])
            outgoing_semaphores =  find_semaphores(object[4])
            relevant_mutexes =  find_mutexes(object[5])
            _activities.append(Activity(object[1], object[2], incoming_semaphores, outgoing_semaphores, relevant_mutexes))
            print(f"Activity created: {object[1]}")
        elif object[0] == "Semaphore":
            _semaphores.append(Semaphore(object[1].strip(), object[2].strip()))
            print(f"Semaphore created: {object[1]}")
        elif object[0] == "Mutex":
            _mutexes.append(Mutex(object[1]))
            print(f"Mutex created: {object[1]}")
    fill_objects(_mutexes)
    fill_objects(_semaphores)
    fill_objects(_tasks)
    return True
    
def fill_objects(objects: List):
    for object in objects:
        if isinstance(object, Task):
            child_activities = object.get_activities()
            for activity in child_activities:
                activity.set_task(object)
        for activity in _activities:
            if isinstance(object, Mutex):
                # Check if the activity has the mutex in its relevant mutexes and check if the mutex is not empty
                relevant_mutexes = [mutex for mutex in activity.get_relevant_mutexes() if mutex]
                if not relevant_mutexes:
                    continue
                activity_list = [mutex for mutex in relevant_mutexes if mutex.get_name() == object.get_name()]
                if activity_list:
                    object.add_to_activity_list(activity)
            if isinstance(object, Semaphore):
                incoming_semaphores = [semaphore for semaphore in activity.get_incoming_semaphores() if semaphore]
                outgoing_semaphores = [semaphore for semaphore in activity.get_outgoing_semaphores() if semaphore]
                if incoming_semaphores:
                    waiting_activites_list = [semaphore for semaphore in incoming_semaphores if semaphore.get_name() == object.get_name()]
                if outgoing_semaphores:
                    actuator_list = [semaphore for semaphore in outgoing_semaphores if semaphore.get_name() == object.get_name()]
                if actuator_list:
                    object.add_to_actuators(activity)
                if waiting_activites_list:
                    object.add_to_waiting_activities(activity)


def find_semaphores(semaphores) -> List:
    found_semaphores = []
    for semaphore in semaphores.split(";"):
        if semaphore.count(":") > 0:
            found_semaphore_relation = []
            # Do something if argument is a list
            for related_semaphore in semaphore.split(":"):
                find_in_array(_semaphores, related_semaphore)[0].set_combined(map(lambda x: find_in_array(_semaphores, x)[0], semaphore.split(':')))
                found_semaphore_relation.extend(find_in_array( _semaphores, related_semaphore.strip()))
            found_semaphores.extend(found_semaphore_relation)
        else:
            # Do something if argument is a string
            found_semaphores.extend(find_in_array(_semaphores, semaphore))
    return found_semaphores
    
def find_mutexes(mutexes) -> List:
    found_mutexes = []
    for mutex in mutexes.split(";"):
        if mutex != "x":
            found_mutexes.extend( find_in_array( _mutexes, mutex))
    return found_mutexes
    
def find_activities(activities) -> List:
    found_activities = []
    for activty in activities.split(";"):
        found_activities.extend( find_in_array( _activities, activty))
    return found_activities

def find_in_array(array, name):
    found_elements = [element for element in array if element._name == name.strip()]
    if len(found_elements) > 0:
        return found_elements
    else:
        print(f"ERROR: No {array[0].__class__.__name__} with name {name} found. Check if the name is correct or the object is missing.")
        return []
    
def data_restructuring(rows: List[List[str]]):
    for row in rows:
        for counter in range(len(row)):
             row[counter] = row[counter].strip()

parse(rows)
print("Finihsed")

Checking structure...
Structure is correct.
Semaphore created: 12
Semaphore created: 13
Semaphore created: 36
Semaphore created: 24
Semaphore created: 46
Semaphore created: 65
Semaphore created: 5ab
Semaphore created: 5ba
Semaphore created: 51
Mutex created: 234
Mutex created: 345
Activity created: 1
Activity created: 2
Activity created: 3
Activity created: 4
Activity created: 5a
Activity created: 5b
Activity created: 6
Task created: 1
Task created: 2
Task created: 3
Task created: 4
Task created: 5
Task created: 6
Finihsed


In [127]:
# https://graphviz.readthedocs.io/en/stable/manual.html

dot = gv.Digraph(comment='TestGraph')

for activity in _activities:
    task = activity.get_task()
    task_name = task.get_name()
    act_name = activity.get_name()

# In der Activity Klasse muss def active mit get_active erstetzt und property entfernt werden!!!
    if activity.get_active():
        dot.node(name=act_name,shape='record', style='filled', fillcolor='green', label='{'+f"{task_name}|{act_name}"+'}')
    dot.node(name=act_name,shape='record', style='filled', fillcolor='white', label='{'+f"{task_name}|{act_name}"+'}')
    pass

bufsemaphores = _semaphores.copy()
while len(bufsemaphores) != 0:
    semaphore = bufsemaphores.pop(0)
    #out_len = len(semaphore.get_actuators())
    #in_len = len(semaphore.get_waiting_activities())
    actuators = semaphore.get_actuators()
    waiting_activities = semaphore.get_waiting_activities()

    # Farben fehlen noch weil wir keine active Variable bei den Semaphoren haben 
    if len(semaphore.get_combined()) == 0:
        if actuators[0].get_task() == waiting_activities[0].get_task():
            dot.edge(f'{actuators[0].get_name()}', f'{waiting_activities[0].get_name()}', label=semaphore.get_state(), arrowhead='onormal', color='black')
        else:
            dot.edge(f'{actuators[0].get_name()}', f'{waiting_activities[0].get_name()}', label=semaphore.get_state(), arrowhead='', color='black')
    else:
        combi = semaphore.get_combined()
        name =  ''
        for obj in combi:
            name += obj.get_name()

        dot.node(name=name, shape='point', width='0.01', height='0.01')
        for obj in combi:
            dot.edge(f'{obj.get_actuators()[0].get_name()}', f'{name}', label=semaphore.get_state(), arrowhead='none', color='black')
            if obj in bufsemaphores:
                bufsemaphores.remove(obj)

        dot.edge(f'{name}', f'{waiting_activities[0].get_name()}', arrowhead='', color='black')

        for mutex in _mutexes:
            dot.node(name=mutex.get_name(), shape='polygon', sides='5', style='filled', fillcolor='white', label=mutex.get_name())
            for activity in mutex.get_activity_list():
                dot.edge(mutex.get_name(), activity.get_name(), style='dashed', arrowhead='none', color='black')

dot.render('testGraph', view=True, format='png')
#svg / png /
os.remove("testGraph")

In [ ]:
# Erstellen der Tasks / Aktivitäten 
# Verbinden durch Semaphoren
# Erstellen der Mutexes und dazugehörigen Verbindungen
#
# Funktion bei Activity zum Ablauf eines Taktes 

In [ ]:
dot = gv.Digraph(comment='TestGraph')

for task in _tasks:
    task_name = task.get_name()
    act_list = ' '.join(task.get_activities())   
    dot.node(name=task_name,shape='record', style='filled', fillcolor='green', label=f"{task_name}|Activity")
    pass



dot.render('TestGraph', view=True, format='svg')
os.remove("TestGraph")